In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp decoder

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python

from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *
import pickle
import os
import torch
import numpy as np
from scipy.stats import spearmanr

In [ ]:
#export
data_path = Path("/home/jupyter/mrdbarros/data/gquest_data/")

In [ ]:
#export
print('Reading data...')
train_data = pd.read_csv(data_path/'train/train.csv')
test_data = pd.read_csv(data_path/'test/test.csv')
sample_submission = pd.read_csv(str(data_path/'sample_submission.csv'))
print('Reading data completed')

Reading data...
Reading data completed


In [ ]:
#export
train_data.columns
target_columns = train_data.columns[11:]
target_columns

Index(['question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written'],
      dtype='object')

In [ ]:
#export
with open(data_path/'vocab.pkl', 'rb') as vocab_file:
 
    # Step 3
    lm_vocab = pickle.load(vocab_file)
 
    # After config_dictionary is read from file

In [ ]:
#export
text_columns=['question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host']


In [ ]:
#train_data=train_data[:100]

In [ ]:
#export
splits = RandomSplitter()(train_data)

In [ ]:
#export
df_tokenized,token_count=tokenize_df(train_data,text_columns)
x_tfms = [attrgetter("text"), Numericalize(vocab=lm_vocab)]


In [ ]:
class DataFrameToTensor(Transform):
    def __init__(self,columns): self.columns=columns
    def encodes(self,o): return tensor(o[self.columns].values.astype(float))
    def decodes(self,x): return x

In [ ]:
dsrc = DataSource(df_tokenized ,splits=splits, tfms=[x_tfms, [DataFrameToTensor(target_columns)]], dl_type=SortedDL)

In [ ]:
#export
torch.cuda.empty_cache()

In [ ]:
#export
dbunch = dsrc.databunch(before_batch=pad_input,bs=16)


In [ ]:
x_train,y_train=dbunch.train_dl.one_batch()
x_valid,y_valid=dbunch.valid_dl.one_batch()
print(type(x_valid))
print(type(y_valid))

<class 'fastai2.text.data.TensorText'>
<class 'torch.Tensor'>


In [ ]:
print(x_train.shape,len(y_train[1]),len(y_train))
print(x_valid.shape,len(y_valid[1]),len(y_valid))
print(x_valid[1].shape)
print(len(y_valid[1]))

torch.Size([16, 10519]) 30 16
torch.Size([24, 3836]) 30 24
torch.Size([3836])
30


In [ ]:
#export
#dbunch.show_batch(max_n=2,trunc_at=60)


In [ ]:
#export
torch.cuda.empty_cache()

In [ ]:
def spearman_corr_coef(a,b):
    full_coef=0.
    #pdb.set_trace()
    n_col=a.shape[1]
    for i in range(n_col):
        #pdb.set_trace()
        coef, p = spearmanr(a[:,i], b[:,i])
        full_coef +=coef
    print("n_col",float(n_col))
    print("full_coef",full_coef)
    return full_coef/float(n_col)

In [ ]:
#export
#learn = text_classifier_learner(dbunch, AWD_LSTM, metrics=[accuracy], path=data_path,drop_mult=0.5)
learn = text_regression_learner(dbunch, AWD_LSTM,seq_len=300, path=data_path,drop_mult=0.5).to_fp16()

In [ ]:
#export
learn = learn.load_encoder('enc1')


In [ ]:
#export
len(target_columns)

30

In [ ]:
#export
#pdb.set_trace()
learn.fit_one_cycle(4, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,time
0,0.249754,0.072040,01:33
1,0.202688,0.054554,01:30
2,0.195247,0.051400,01:33
3,0.192770,0.050802,01:31


In [ ]:
learn.save("first_decoder2")

In [ ]:
learn.load("first_decoder")

In [ ]:
learn.unfreeze()
learn.opt = learn.create_opt()
learn.fit_one_cycle(8, slice(1e-5,1e-3))

epoch,train_loss,valid_loss,time
0,0.127326,0.050574,01:41
1,0.062126,0.051281,01:39
2,0.054993,0.049022,01:50
3,0.051777,0.048527,01:44
4,0.049745,0.046926,01:42
5,0.049219,0.046863,01:49
6,0.049266,0.046299,01:41
7,0.048961,0.046325,01:50


In [ ]:
a,b=learn.get_preds()


torch.Size([1215, 30])

In [ ]:
a.shape

In [ ]:
spearman_corr_coef(a,b)

n_col 30.0
full_coef 9.442676832626423


0.31475589442088076

In [ ]:
print(full_target[:,0])
print(full_predict[:,0])

tensor([1.0000, 1.0000, 0.5556, 0.8889, 0.8889, 1.0000, 0.8889, 1.0000, 1.0000,
        0.8889, 1.0000, 0.8889, 0.8889, 0.8333, 1.0000, 1.0000, 0.8889, 1.0000,
        1.0000, 0.7778])
tensor([-0.2206,  0.6313,  0.0508,  0.7026, -0.1221,  0.3962, -0.3396,  0.1775,
        -0.4016,  0.5615,  0.0323,  0.5073,  0.7290,  0.5127,  0.3516,  0.3015,
         0.2112,  0.6333,  0.2668,  0.1365])


In [ ]:
spearman_corr_coef(full_predict,full_target)

In [ ]:
full_predict[0]

tensor([-0.1260, -0.0189, -0.0305,  0.0282,  0.1493, -0.0967,  0.1312,  0.0971,
         0.2717, -0.0007, -0.2765,  0.3127,  0.0387,  0.0097, -0.0135,  0.0609,
        -0.3462,  0.1779,  0.3245, -0.0028,  0.0162, -0.0771, -0.0212, -0.0601,
        -0.0312, -0.0744, -0.4155,  0.1605,  0.4971,  0.0330])

In [ ]:

learn.predict(L(full_x))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()


In [ ]:
#export
@delegates(Learner.__init__)
def text_regression_learner(dbunch, arch, seq_len=72, config=None, pretrained=True, drop_mult=0.5,
                            lin_ftrs=None, ps=None, **kwargs):
    "Create a `Learner` with a text classifier from `data` and `arch`."
    vocab = _get_text_vocab(dbunch)
    model = get_text_classifier(arch, len(vocab), len(dbunch.train_ds[0][1]), seq_len=seq_len, config=config,
                                drop_mult=drop_mult, lin_ftrs=lin_ftrs, ps=ps)
    meta = _model_meta[arch]
    learn = TextLearner(dbunch, model, loss_func=MSELossFlat(), splitter=meta['split_clas'], **kwargs)
    if pretrained:
        if 'url' not in meta:
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], c_key='model')
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn = learn.load_pretrained(*fnames, model=learn.model[0])
        learn.freeze()
    return learn